# Simulating Strong Gravitational Lenses
In this notebook, we go through the method of simulating strong gravitational lenses with JWST and Euclid observation configurations using [lenstronomy](https://lenstronomy.readthedocs.io/en/latest/).
The goal is to generate a dataset of lenses in each JWST NIRcam filter and Euclid-VIS and Euclid-NISP filters to train the cGAN against. For more examples of simulating lenses, especially using different observation configurations, I recommend: https://github.com/lenstronomy/lenstronomy-tutorials.

In [1]:
# standard imports
import copy
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import os

This notebook requires working with spectroscopic data, so, you will need to install [Speclite](https://speclite.readthedocs.io/en/latest/) and Skypy.

In [2]:
# skypy imports
import skypy
from skypy.pipeline import Pipeline
import speclite
import astropy
from astropy.modeling import models

Since Lenstronomy does not support all 6 JWST NIRcam bands, you will need to change the observation config for JWST. This is done in *JWST_Config.py* and imported below.

In [4]:
# lenstronomy module imports
from lenstronomy.SimulationAPI.sim_api import SimAPI
import lenstronomy.Plots.plot_util as plot_util
from lenstronomy.Util import constants
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
import lenstronomy.Util.data_util as data_util
import lenstronomy.Util.util as util
from lenstronomy.LightModel.Profiles.gaussian import GaussianEllipse
from JWST_Config import JWST
from astropy.io import fits
from skimage.transform import resize

Lenstronomy requires a PSF model for the JWST NIRcam filters. These can be downloaded from here: https://stsci.app.box.com/v/jwst-simulated-psf-library.
Below, we also load in the NIRcam filter response curves for Speclite. These are generated in the *JWST_Filters.ipynb*.

In [5]:
# path to YOUR working directory
wdir = '/Users/ruby/Documents/Python Scripts/cGAN/Euclid-JWST/'

# path to YOUR downloaded PSF models
psf_dir = wdir+'PSFs/'

# get the filter information for Speclite
filters_dir = wdir+'/FilterInformation/'

# PSF models
psfs = ['PSF_NIRcam_in_flight_opd_filter_F115W.fits',
        'PSF_NIRcam_in_flight_opd_filter_F150W.fits',
        'PSF_NIRcam_in_flight_opd_filter_F200W.fits',
        'PSF_NIRcam_in_flight_opd_filter_F277W.fits',
        'PSF_NIRcam_in_flight_opd_filter_F356W.fits',
        'PSF_NIRcam_in_flight_opd_filter_F444W.fits'
        ]

The PSF requires an odd kernel so we resize the PSF models.

In [6]:
f115w_psf = fits.getdata(psf_dir+psfs[0])
f115w_psf_array = resize(f115w_psf, ((f115w_psf.shape[0]+1, f115w_psf.shape[1]+1)))
f150w_psf = fits.getdata(psf_dir+psfs[1])
f150w_psf_array = resize(f150w_psf, ((f150w_psf.shape[0]+1, f150w_psf.shape[1]+1)))
f200w_psf = fits.getdata(psf_dir+psfs[2])
f200w_psf_array = resize(f200w_psf, ((f200w_psf.shape[0]+1, f200w_psf.shape[1]+1)))
f277w_psf = fits.getdata(psf_dir+psfs[3])
f277w_psf_array = resize(f277w_psf, ((f277w_psf.shape[0]+1, f277w_psf.shape[1]+1)))
f356w_psf = fits.getdata(psf_dir+psfs[4])
f356w_psf_array = resize(f356w_psf, ((f356w_psf.shape[0]+1, f356w_psf.shape[1]+1)))
f444w_psf = fits.getdata(psf_dir+psfs[5])
f444w_psf_array = resize(f444w_psf, ((f444w_psf.shape[0]+1, f444w_psf.shape[1]+1)))
psf_models = [f115w_psf_array, f150w_psf_array, f200w_psf_array, f277w_psf_array,
              f356w_psf_array, f444w_psf_array]


Get the NIRcam filter response curves for Speclite.

In [7]:
# get the nircam filters
f115w = os.path.join(filters_dir, 'NIRcam-F115W.ecsv')
f150w = os.path.join(filters_dir, 'NIRcam-F150W.ecsv')
f200w = os.path.join(filters_dir, 'NIRcam-F200W.ecsv')
f277w = os.path.join(filters_dir, 'NIRcam-F277W.ecsv')
f356w = os.path.join(filters_dir, 'NIRcam-F356W.ecsv')
f444w = os.path.join(filters_dir, 'NIRcam-F444W.ecsv')
# and pass to speclite to 'load_filters'
nircam_f115w = speclite.filters.load_filter(f115w)
nircam_f150w = speclite.filters.load_filter(f150w)
nircam_f200w = speclite.filters.load_filter(f200w)
nircam_f277w = speclite.filters.load_filter(f277w)
nircam_f356w = speclite.filters.load_filter(f356w)
nircam_f444w = speclite.filters.load_filter(f444w)
nircam_filters = [nircam_f115w, nircam_f150w, nircam_f200w, nircam_f277w, nircam_f356w, nircam_f444w]

Our strong lensing simulations are based on the lens parameters from http://metcalf1.difa.unibo.it/blf-portal/gg_challenge.html. You will need to download the image catalog and save as a csv file. Below, we load in the csv file.

In [8]:
# load in the log lens parameters
csv = pd.read_csv(psf_dir+'image_catalog2.0train.csv')

Define the cosmology we are using. We impose a Flat Lambda CDM cosmology.

In [9]:
# define the cosmology
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.)

We will need to use the Faber-Jackson magnitude to mass relationship to calculate the velocity dispersion in our simulations. we use magnitude and redshift pairs from the single lens dataset. We load the data from Table 1 in https://articles.adsabs.harvard.edu/pdf/1976ApJ...204..668F. We also give the csv file in the *Files* directory.

In [10]:
def give_me_a_straight_line(x,y):
        w, b  = np.polyfit(x,y,deg=1)
        return w,b 

def MakeFaberJackson():
    FJG = pd.read_csv(wdir+'FaberJacksonGraph.csv')

    X = FJG['MB (mag)'].values
    Y = np.log10(FJG['Fourier Velocity 1'].values)

    w, b = give_me_a_straight_line(X,Y)

    return X,Y,w,b
    #y = w * X + b --> calculates velocity dispersion from magnitude

Now, we get the CCD specifications for Euclid. The information for the Euclid camera and band_obs are taken from Lenstronomy. Small changes are made to make the compound lens dataset as similiar to the single lens dataset as possible.

In [11]:
# now get the Euclid band observations
gauss = GaussianEllipse()
Euclid_camera_NISP = {'read_noise': 10,  # std of noise generated by read-out (in units of electrons)
               'pixel_scale': 0.3,  # scale (in arcseonds) of pixels
               'ccd_gain': 4.5  # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
              }

Euclid_camera_VIS = {'read_noise': 4.2,  # std of noise generated by read-out (in units of electrons)
               'pixel_scale': 0.1,  # scale (in arcseonds) of pixels
               'ccd_gain': 3.1  # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
              }

Euclid_J_band_obs = {'exposure_time': 264.,  # exposure time per image (in seconds)
                   'sky_brightness': 22.35,  # sky brightness (in magnitude per square arcseconds)
                   'magnitude_zero_point': 24.0,  # magnitude in which 1 count per second per arcsecond square is registered (in ADU's)
                   'num_exposures': 3,  # number of exposures that are combined
                   'seeing': 0.16,  # full width at half maximum of the PSF (if not specific psf_model is specified)
                   'psf_type': 'GAUSSIAN',  # string, type of PSF ('GAUSSIAN' and 'PIXEL' supported)
                   'kernel_point_source': None  # 2d numpy array, model of PSF centered with odd number of pixels per axis (optional when psf_type='PIXEL' is chosen)
                  }

Euclid_Y_band_obs = {'exposure_time': 264.,
                   'sky_brightness': 22.35,
                   'magnitude_zero_point': 24.0,
                   'num_exposures': 3,
                   'seeing': 0.16,
                   'psf_type': 'GAUSSIAN'}

Euclid_H_band_obs = {'exposure_time': 264.,
                   'sky_brightness': 22.35,
                   'magnitude_zero_point': 24.0,
                   'num_exposures': 3,
                   'seeing': 0.16,
                   'psf_type': 'GAUSSIAN'}

Euclid_VIS_band_obs = {'exposure_time': 1800.,
                   'sky_brightness': 22.35,
                   'magnitude_zero_point': 24.0,
                   'num_exposures': 3,
                   'seeing': 0.16,
                   'psf_type': 'GAUSSIAN'}

We merge the band observations and instrument for the 4 bands.

In [12]:
kwargs_J_band = util.merge_dicts(Euclid_camera_NISP, Euclid_J_band_obs)
kwargs_Y_band = util.merge_dicts(Euclid_camera_NISP, Euclid_Y_band_obs)
kwargs_H_band = util.merge_dicts(Euclid_camera_NISP, Euclid_H_band_obs)
kwargs_VIS_band = util.merge_dicts(Euclid_camera_VIS, Euclid_VIS_band_obs)
kwargs_Euclid = [kwargs_J_band, kwargs_Y_band, kwargs_H_band, kwargs_VIS_band]

We'll do the same for the JWST NIRcam bands.

In [13]:
# JWST band observations
F115W = JWST(band='F115W', psf_type='PIXEL', coadd_years=None)
F150W = JWST(band='F150W', psf_type='PIXEL', coadd_years=None)
F200W = JWST(band='F200W', psf_type='PIXEL', coadd_years=None)
F277W = JWST(band='F277W', psf_type='PIXEL', coadd_years=None)
F356W = JWST(band='F356W', psf_type='PIXEL', coadd_years=None)
F444W = JWST(band='F444W', psf_type='PIXEL', coadd_years=None)
jwst_sw = [F115W, F150W, F200W] 
jwst_lw = [F277W, F356W, F444W]

# kwargs for each band
kwargs_f115W = util.merge_dicts(F115W.kwargs_single_band(), dict(kernel_point_source=psf_models[0]))
kwargs_f150W = util.merge_dicts(F150W.kwargs_single_band(), dict(kernel_point_source=psf_models[1]))
kwargs_f200W = util.merge_dicts(F200W.kwargs_single_band(), dict(kernel_point_source=psf_models[2]))
kwargs_f277W = util.merge_dicts(F277W.kwargs_single_band(), dict(kernel_point_source=psf_models[3]))
kwargs_f356W = util.merge_dicts(F356W.kwargs_single_band(), dict(kernel_point_source=psf_models[4]))
kwargs_f444W = util.merge_dicts(F444W.kwargs_single_band(), dict(kernel_point_source=psf_models[5]))
kwargs_NIRcam = [kwargs_f115W, kwargs_f150W, kwargs_f200W, kwargs_f277W, kwargs_f356W, kwargs_f444W]

### Making Simulated Compound Lenses
Here, we define the gravitational lensing process. The function below generates the compound lens system with the parameters provided to it for both Euclid and JWST NIRcam band observations. This returns the lens image for each individual JWST NIRcam filter and for all 4 Euclid bands, along with 4 stacked images:
1. a 3-channel image for Euclid-VIS
2. an RGB image for Euclid-NISP (RGB --> NISP-H, NISP-Y, NISP-J)
3. an RGB image for JWST NIRcam short wavelength (RGB --> F200W, F150W, F115W)
4. an RGB image for JWST NIRcam long wavelength (RGB --> F444W, F356W, F277W).

In [14]:
# define variables
numPix = 64
kwargs_numerics = {'point_source_supersampling_factor': 1}

# function to make the compound lenses given the lenstronomy kwargs
def CompoundLensMaker2(kwargs_model_composite, kwargs_lens, kwargs_lens_light_mag_F115W, kwargs_source_mag_F115W,
                       kwargs_lens_light_mag_F150W, kwargs_source_mag_F150W, kwargs_lens_light_mag_F200W, 
                      kwargs_source_mag_F200W, kwargs_lens_light_mag_F277W, kwargs_source_mag_F277W,
                      kwargs_lens_light_mag_F356W, kwargs_source_mag_F356W, kwargs_lens_light_mag_F444W,
                      kwargs_source_mag_F444W, kwargs_NIRcam, kwargs_lens_light_mag_VIS, kwargs_source_mag_VIS,
                      kwargs_lens_light_mag_J, kwargs_source_mag_J, kwargs_lens_light_mag_Y, kwargs_source_mag_Y,
                      kwargs_lens_light_mag_H, kwargs_source_mag_H, kwargs_Euclid):
    
    # we only simulate here the g-band, see above how to combine multiple bands
    kwargs_numerics = {'point_source_supersampling_factor': 1}
    kwargs_ps_mag_F115W = [{'magnitude': 13, 'ra_source': 0.03, 'dec_source': -30}]
    kwargs_ps_mag_VIS = [{'magnitude': 13, 'ra_source': 0.03, 'dec_source': -30}]
    
    # start with the JWST NIRcam kwargs
    sim_F115W = SimAPI(numpix=numPix, kwargs_single_band=kwargs_NIRcam[0], kwargs_model=kwargs_model_composite)
    imSim_F115W = sim_F115W.image_model_class(kwargs_numerics)
    sim_F150W = SimAPI(numpix=numPix, kwargs_single_band=kwargs_NIRcam[1], kwargs_model=kwargs_model_composite)    
    imSim_F150W = sim_F150W.image_model_class(kwargs_numerics)
    sim_F200W = SimAPI(numpix=numPix, kwargs_single_band=kwargs_NIRcam[2], kwargs_model=kwargs_model_composite)
    imSim_F200W = sim_F200W.image_model_class(kwargs_numerics)
    sim_F277W = SimAPI(numpix=numPix, kwargs_single_band=kwargs_NIRcam[3], kwargs_model=kwargs_model_composite)    
    imSim_F277W = sim_F277W.image_model_class(kwargs_numerics)
    sim_F356W = SimAPI(numpix=numPix, kwargs_single_band=kwargs_NIRcam[4], kwargs_model=kwargs_model_composite)
    imSim_F356W = sim_F356W.image_model_class(kwargs_numerics)
    sim_F444W = SimAPI(numpix=numPix, kwargs_single_band=kwargs_NIRcam[5], kwargs_model=kwargs_model_composite)    
    imSim_F444W = sim_F444W.image_model_class(kwargs_numerics)

# F150W-band
    f115w_f150w_ps = 0
    kwargs_ps_mag_F150W = copy.deepcopy(kwargs_ps_mag_F115W)
    kwargs_ps_mag_F150W[0]['magnitude'] -= f115w_f150w_ps
    
# F200W-band
    f115w_f200w_ps = 0
    kwargs_ps_mag_F200W = copy.deepcopy(kwargs_ps_mag_F115W)
    kwargs_ps_mag_F200W[0]['magnitude'] -= f115w_f200w_ps
    
# F277W band
    f115w_f277w_ps = 0
    kwargs_ps_mag_F277W = copy.deepcopy(kwargs_ps_mag_F115W)
    kwargs_ps_mag_F277W[0]['magnitude'] -= f115w_f277w_ps
    
# F277W-F356W
    f115w_f356w_ps = 0
    kwargs_ps_mag_F356W = copy.deepcopy(kwargs_ps_mag_F115W)
    kwargs_ps_mag_F356W[0]['magnitude'] -= f115w_f356w_ps
    
# F277W-F356W
    f115w_f444w_ps = 0
    kwargs_ps_mag_F444W = copy.deepcopy(kwargs_ps_mag_F115W)
    kwargs_ps_mag_F444W[0]['magnitude'] -= f115w_f444w_ps

# turn magnitude kwargs into lenstronomy kwargs
    kwargs_lens_light_F115W, kwargs_source_F115W, kwargs_ps_F115W = sim_F115W.magnitude2amplitude(kwargs_lens_light_mag_F115W, kwargs_source_mag_F115W, kwargs_ps_mag_F115W)
    kwargs_lens_light_F150W, kwargs_source_F150W, kwargs_ps_F150W = sim_F150W.magnitude2amplitude(kwargs_lens_light_mag_F150W, kwargs_source_mag_F150W, kwargs_ps_mag_F150W)
    kwargs_lens_light_F200W, kwargs_source_F200W, kwargs_ps_F200W = sim_F200W.magnitude2amplitude(kwargs_lens_light_mag_F200W, kwargs_source_mag_F200W, kwargs_ps_mag_F200W)
    kwargs_lens_light_F277W, kwargs_source_F277W, kwargs_ps_F277W = sim_F277W.magnitude2amplitude(kwargs_lens_light_mag_F277W, kwargs_source_mag_F277W, kwargs_ps_mag_F277W)
    kwargs_lens_light_F356W, kwargs_source_F356W, kwargs_ps_F356W = sim_F356W.magnitude2amplitude(kwargs_lens_light_mag_F356W, kwargs_source_mag_F356W, kwargs_ps_mag_F356W)
    kwargs_lens_light_F444W, kwargs_source_F444W, kwargs_ps_F444W = sim_F444W.magnitude2amplitude(kwargs_lens_light_mag_F444W, kwargs_source_mag_F444W, kwargs_ps_mag_F444W)
    
    image_F115W = imSim_F115W.image(kwargs_lens, kwargs_source_F115W, kwargs_lens_light_F115W)
    image_F150W = imSim_F150W.image(kwargs_lens, kwargs_source_F150W, kwargs_lens_light_F150W)
    image_F200W = imSim_F200W.image(kwargs_lens, kwargs_source_F200W, kwargs_lens_light_F200W)
    image_F277W = imSim_F277W.image(kwargs_lens, kwargs_source_F277W, kwargs_lens_light_F277W)
    image_F356W = imSim_F356W.image(kwargs_lens, kwargs_source_F356W, kwargs_lens_light_F356W)
    image_F444W = imSim_F444W.image(kwargs_lens, kwargs_source_F444W, kwargs_lens_light_F444W)
    # add noise 
    image_F115W += sim_F115W.noise_for_model(model=image_F115W)
    image_F150W += sim_F150W.noise_for_model(model=image_F150W)
    image_F200W += sim_F200W.noise_for_model(model=image_F200W)
    image_F277W += sim_F277W.noise_for_model(model=image_F277W)
    image_F356W += sim_F356W.noise_for_model(model=image_F356W)
    image_F444W += sim_F444W.noise_for_model(model=image_F444W)
    
    # now the Euclid kwargs
    sim_VIS = SimAPI(numpix=numPix, kwargs_single_band=kwargs_Euclid[3], kwargs_model=kwargs_model_composite)
    imSim_VIS = sim_VIS.image_model_class(kwargs_numerics)
    sim_J = SimAPI(numpix=numPix, kwargs_single_band=kwargs_Euclid[0], kwargs_model=kwargs_model_composite)    
    imSim_J = sim_J.image_model_class(kwargs_numerics)
    sim_Y = SimAPI(numpix=numPix, kwargs_single_band=kwargs_Euclid[1], kwargs_model=kwargs_model_composite)
    imSim_Y = sim_Y.image_model_class(kwargs_numerics)
    sim_H = SimAPI(numpix=numPix, kwargs_single_band=kwargs_Euclid[2], kwargs_model=kwargs_model_composite)
    imSim_H = sim_H.image_model_class(kwargs_numerics)

    # Y-band
    V_Y_ps = 0
    kwargs_ps_mag_Y = copy.deepcopy(kwargs_ps_mag_VIS)
    kwargs_ps_mag_Y[0]['magnitude'] -= V_Y_ps

    # J-band
    V_J_ps = 0
    kwargs_ps_mag_J = copy.deepcopy(kwargs_ps_mag_VIS)
    kwargs_ps_mag_J[0]['magnitude'] -= V_J_ps

    # H-band
    V_H_ps = 0
    kwargs_ps_mag_H = copy.deepcopy(kwargs_ps_mag_VIS)
    kwargs_ps_mag_H[0]['magnitude'] -= V_H_ps

    # turn magnitude kwargs into lenstronomy kwargs
    kwargs_lens_light_VIS, kwargs_source_VIS, kwargs_ps_VIS = sim_VIS.magnitude2amplitude(kwargs_lens_light_mag_VIS, kwargs_source_mag_VIS, kwargs_ps_mag_VIS)
    kwargs_lens_light_Y, kwargs_source_Y, kwargs_ps_Y = sim_Y.magnitude2amplitude(kwargs_lens_light_mag_Y, kwargs_source_mag_Y, kwargs_ps_mag_Y)
    kwargs_lens_light_J, kwargs_source_J, kwargs_ps_J = sim_J.magnitude2amplitude(kwargs_lens_light_mag_J, kwargs_source_mag_J, kwargs_ps_mag_J)
    kwargs_lens_light_H, kwargs_source_H, kwargs_ps_H = sim_H.magnitude2amplitude(kwargs_lens_light_mag_H, kwargs_source_mag_H, kwargs_ps_mag_H)
    
    image_VIS = imSim_VIS.image(kwargs_lens, kwargs_source_VIS, kwargs_lens_light_VIS)
    image_J = imSim_J.image(kwargs_lens, kwargs_source_J, kwargs_lens_light_J)
    image_Y = imSim_Y.image(kwargs_lens, kwargs_source_Y, kwargs_lens_light_Y)
    image_H = imSim_H.image(kwargs_lens, kwargs_source_H, kwargs_lens_light_H)
    # add noise  
    image_VIS += sim_VIS.noise_for_model(model=image_VIS)
    image_J += sim_J.noise_for_model(model=image_J)
    image_Y += sim_Y.noise_for_model(model=image_Y)
    image_H += sim_H.noise_for_model(model=image_H)

    # JWST NIRcam plots
    img_SW = np.zeros((64, 64, 3), dtype=float) # NIRcam SW
    img_LW= np.zeros((64, 64, 3), dtype=float)  # NIRcam LW

    img_SW[:,:,0] = plot_util.sqrt(image_F200W, scale_min=0, scale_max=10000)                                  
    img_SW[:,:,1] = plot_util.sqrt(image_F150W, scale_min=0, scale_max=10000) 
    img_SW[:,:,2] = plot_util.sqrt(image_F115W, scale_min=0, scale_max=10000)                                  

    img_LW[:,:,0] = plot_util.sqrt(image_F444W, scale_min=0, scale_max=10000)                                   
    img_LW[:,:,1] = plot_util.sqrt(image_F356W, scale_min=0, scale_max=10000)                                  
    img_LW[:,:,2] = plot_util.sqrt(image_F277W, scale_min=0, scale_max=10000)                                  

    # Euclid VIS and NISP plots
    img_VIS = np.zeros((64, 64, 1), dtype=float)
    img_NISP = np.zeros((64, 64, 3), dtype=float)
    img_VIS = plot_util.sqrt(image_VIS, scale_min=0, scale_max=1000)
    img_NISP[:,:,0] = plot_util.sqrt(image_Y, scale_min=0, scale_max=1000)
    img_NISP[:,:,1] = plot_util.sqrt(image_J, scale_min=0, scale_max=1000)
    img_NISP[:,:,2] = plot_util.sqrt(image_H, scale_min=0, scale_max=1000)
    return img_SW, img_LW, image_F115W, image_F150W, image_F200W, image_F277W, image_F356W, image_F444W, img_VIS, img_NISP, image_VIS, image_J, image_Y, image_H


### Make the Galaxies
We'll produce the source and lens galaxies by using the redshift and magnitude pairs from the single lens dataset. The apparent magnitude for each band is generated using SkyPy. This is used to make Blue source galaxies and Red lens galaxies. We'll need to load the JWST NIRcam filter responses for Speclite.

In [15]:
# making blue and red galaxy pairs for JWST
def MakeBlueGalaxies_JWST(redshift, M_lens, cosmology):
    # the following values are taken from Skypy source code
    alpha0 = [2.079, 3.524, 1.917, 1.992, 2.536]
    alpha1 = [2.265, 3.862, 1.921, 1.685, 2.480]
    weight = [3.47e+09, 3.31e+06, 2.13e+09, 1.64e+10, 1.01e+09]
    spectral_coefficients = skypy.galaxies.spectrum.dirichlet_coefficients(redshift, alpha0, alpha1, weight)
    filters = 'NIRcam-F115W', 'NIRcam-F150W', 'NIRcam-F200W', 'NIRcam-F277W', 'NIRcam-F356W', 'NIRcam-F444W'
    M = M_lens - cosmo.distmod(redshift).value
    stellar_mass = skypy.galaxies.spectrum.kcorrect.stellar_mass(spectral_coefficients, M, filters)
    app_mag = skypy.galaxies.spectrum.kcorrect.apparent_magnitudes(spectral_coefficients, redshift, filters, cosmology, stellar_mass=stellar_mass)
    return app_mag

def MakeRedGalaxies_JWST(redshift, M_lens, cosmology):
    # the following values are taken from the Skypy source code
    alpha0 = [2.461, 2.358, 2.568, 2.268, 2.402]
    alpha1 = [2.410, 2.340, 2.200, 2.540, 2.464]
    weight = [3.84e+09, 1.57e+06, 3.91e+08, 4.66e+10, 3.03e+07]
    spectral_coefficients = skypy.galaxies.spectrum.dirichlet_coefficients(redshift, alpha0, alpha1, weight)
    filters = 'NIRcam-F115W', 'NIRcam-F150W', 'NIRcam-F200W', 'NIRcam-F277W', 'NIRcam-F356W', 'NIRcam-F444W'
    M = M_lens - cosmo.distmod(redshift).value
    stellar_mass = skypy.galaxies.spectrum.kcorrect.stellar_mass(spectral_coefficients, M, filters)
    app_mag = skypy.galaxies.spectrum.kcorrect.apparent_magnitudes(spectral_coefficients, redshift, filters, cosmology, stellar_mass=stellar_mass)
    return app_mag

We'll use the same functions for Euclid but with the Euclid filter response curves, already supported by Speclite.

In [16]:
# making red and blue galaxies for Euclid VIS and NISP
def MakeBlueGalaxies_Euclid(redshift, M_lens, cosmology):
    # the following values are taken from the Skypy source code
    alpha0= [2.079, 3.524, 1.917, 1.992, 2.536]
    alpha1= [2.265, 3.862, 1.921, 1.685, 2.480]
    weight= [3.47e+09, 3.31e+06, 2.13e+09, 1.64e+10, 1.01e+09]
    spectral_coefficients = skypy.galaxies.spectrum.dirichlet_coefficients(redshift, alpha0,alpha1,weight)
    filters = 'Euclid-VIS','Euclid-Y','Euclid-H','Euclid-J'
    M = M_lens - cosmo.distmod(redshift).value
    stellar_mass = skypy.galaxies.spectrum.kcorrect.stellar_mass(spectral_coefficients,M,filters)
    app_mag = skypy.galaxies.spectrum.kcorrect.apparent_magnitudes( spectral_coefficients, redshift, filters,cosmology, stellar_mass=stellar_mass)
    
    return app_mag

def MakeRedGalaxies_Euclid(redshift, M_lens, cosmology):
    # the following values are taken from the Skypy source code.
    alpha0= [2.461, 2.358, 2.568, 2.268, 2.402]
    alpha1= [2.410, 2.340, 2.200, 2.540, 2.464]
    weight = [3.84e+09, 1.57e+06, 3.91e+08, 4.66e+10, 3.03e+07]
    spectral_coefficients = skypy.galaxies.spectrum.dirichlet_coefficients(redshift, alpha0,alpha1,weight)
    filters = 'Euclid-VIS','Euclid-Y','Euclid-H','Euclid-J'
    M = M_lens - cosmo.distmod(redshift).value
    stellar_mass = skypy.galaxies.spectrum.kcorrect.stellar_mass(spectral_coefficients,M,filters)
    app_mag = skypy.galaxies.spectrum.kcorrect.apparent_magnitudes( spectral_coefficients, redshift, filters, cosmology, stellar_mass=stellar_mass)
    
    return app_mag

### Magnitude and Redshift Pairs
To calaculate the velocity dispersion using the Faber-Jackson relation, we'll require magnitude-redshift pairs. The following function randomly selects magnitude and redshift pairs from the single lens dataset. The lens galaxies are arranged to ensure that the lenses appear in the correct redshift order.

In [17]:
# sampling magnitudes and redshift
def Generate_Lens_System_Mag_Z_2():
    index2 = np.random.randint(0,100000)

    Z3_Data_A = csv[csv['z_source']>3]
    Z3_Data_L = csv[csv['z_source']<3]

    index0 = np.random.randint(0,Z3_Data_A.shape[0])
    index1 = np.random.randint(0,Z3_Data_L.shape[0])
    
    z_source = Z3_Data_A['z_source'].values[index0]
    mag_source = Z3_Data_A['mag_source'].values[index0]
    
    Z_lens_1 = Z3_Data_L['z_source'].values[index1]
    mag_lens_1 = Z3_Data_L['mag_source'].values[index1]

    Z_lens_2 = csv['z_lens'].values[index2]
    mag_lens_2 = csv['mag_lens'].values[index2]
    
    if Z_lens_1 > Z_lens_2: 
        Mid_lens_Z = Z_lens_1
        Mid_lens_Mag = mag_lens_1
        Edge_lens_Z = Z_lens_2
        Edge_lens_Mag = mag_lens_2
    else:
        Mid_lens_Z = Z_lens_2
        Mid_lens_Mag = mag_lens_2
        Edge_lens_Z = Z_lens_1
        Edge_lens_Mag = mag_lens_1
        
    return z_source, mag_source, Mid_lens_Z, Mid_lens_Mag, Edge_lens_Z, Edge_lens_Mag

### Einstein Radius
We'll use the computed velocity dispersion from the Faber-Jackson relationship to calculate the Einstein radius, $\theta_{E}$, of a gravitational lens with the following function.

In [18]:
# get the velocity dispersion from the Faber Jackson Graph
def Get_Theta_E_From_Velo_Dist(mag_lens, Z_lens, z_source):
    # get the Faber-Jackson Graph
    X,Y,w,b = MakeFaberJackson()
    log_vel_d = w * (mag_lens - cosmo.distmod(Z_lens).value) + b
    vel_disp_iso =10**log_vel_d
    lens_cosmo = LensCosmo(Z_lens, z_source, cosmo=cosmo)
    theta_E_SIS = lens_cosmo.sis_sigma_v2theta_E(vel_disp_iso)
    
    return theta_E_SIS

### Position of the Lensing Galaxy
With the computed velocity dispersion and Einstein radius, we can calculate the position of the lensing galaxy. The function below generates the position of a galaxy within the image when given an Einstein Radius and $x$- and $y$-coordinates.

In [19]:
def generateLensPos(X,Y, E_theta):
    x1 = np.random.uniform(-E_theta/2, E_theta/2)
    y1 = np.random.uniform(-E_theta/2, E_theta/2)
    
    if X + x1 >=5 or X + x1 <= -5:
        X_lens = X - x1
    else:
        X_lens = X + x1
        
    if Y + y1 >=5 or Y + y1 <= -5:
        Y_lens = Y - y1
    else:
        Y_lens = Y + y1
        
    return X_lens, Y_lens

### Source Rotation
This following function converts the angle and the axis ratio of the galaxy into ellipticity components, $e_{1}$ and $e_{2}$.

In [20]:
def phi_q2_ellipticity(phi, q):
    e1 = (1 - q) / (1 + q) * np.cos(2 * phi)
    e2 = (1 - q) / (1 + q) * np.sin(2 * phi)    
    
    return e1, e2

### Generating the Lens Dataset
This process generates everything needed to make the compound lenses.

The following function gets three magnitude and redshift pairs for the galaxies in the image. The parameters returned by the functions above are computed and sampled from for each variable required to make the compound lenses.
If an error occurs, every variable is assigned 9999 and *countMod=0*. This way, we can recognise the error.
The function returns JWST NIRcam short wavelength and long wavelength band images, individual NIRcam band images, the Euclid-VIS and Euclid_NISP images along with individual NISP-J, NISP-Y, NISP-H images, the parameters used to generate the lenses and finally, the *countMod* variable.


In [23]:
# now generate the compound lenses
def CreateCompoundLenses(cosmo=cosmo):
    Edge_source_Z, Edge_source_Mag, Mid_lens_Z, Mid_lens_Mag, Edge_lens_Z, Edge_lens_Mag = Generate_Lens_System_Mag_Z_2()
    
    try:
        Edge_Lens_Gal_Mag_JWST = MakeRedGalaxies_JWST(Edge_lens_Z,Edge_lens_Mag, cosmo)
        Mid_Lens_Gal_Mag_JWST = MakeRedGalaxies_JWST(Mid_lens_Z,Mid_lens_Mag, cosmo)
        Edge_Source_Gal_Mag_JWST = MakeBlueGalaxies_JWST(Edge_source_Z, Edge_source_Mag, cosmo)
    
        Edge_Lens_Gal_Mag_Euclid = MakeRedGalaxies_Euclid(Edge_lens_Z,Edge_lens_Mag, cosmo)
        Mid_Lens_Gal_Mag_Euclid = MakeRedGalaxies_Euclid(Mid_lens_Z,Mid_lens_Mag, cosmo)
        Edge_Source_Gal_Mag_Euclid = MakeBlueGalaxies_Euclid(Edge_source_Z, Edge_source_Mag, cosmo)

        theta_E_SIS_Mid = Get_Theta_E_From_Velo_Dist(Mid_lens_Mag, Mid_lens_Z, Edge_source_Z)
        theta_E_SIS_EdgeML =  Get_Theta_E_From_Velo_Dist(Edge_lens_Mag, Edge_lens_Z, Mid_lens_Z)
        theta_E_SIS_EdgeES = Get_Theta_E_From_Velo_Dist(Edge_lens_Mag, Edge_lens_Z, Edge_source_Z)

        R_sersic_Edge_Source = np.random.uniform(0.2, 7)
        n_sersic_Edge_Source = np.random.uniform(1.5, 4)
        centre_X_Edge_Source = 0 
        centre_Y_Edge_Source = 0 

        R_sersic_Mid_Lens = np.random.uniform(0.2, 7)
        n_sersic_Mid_Lens = np.random.uniform(1.5, 4)
        phi_Mid_Lens = np.random.uniform(0, np.pi)
        q_Mid_Lens =  np.random.uniform(0.3, 1)
        e1_Mid_Lens , e2_Mid_Lens = phi_q2_ellipticity(phi_Mid_Lens, q_Mid_Lens)
        centre_X_Mid_Lens = centre_X_Edge_Source + np.random.uniform(-0.25, 0.25)
        centre_Y_Mid_Lens = centre_Y_Edge_Source + np.random.uniform(-0.25, 0.25)
        gamma1_Mid_Lens =  np.random.uniform(-0.05, 0.05)
        gamma2_Mid_Lens =  np.random.uniform(-0.05, 0.05)

        R_sersic_Edge_Lens = np.random.uniform(0.2, 7)
        n_sersic_Edge_Lens = np.random.uniform(1.5, 4)
        phi_Edge_Lens =  np.random.uniform(0, np.pi)
        q_Edge_Lens =   np.random.uniform(0.3, 1)
        e1_Edge_Lens , e2_Edge_Lens =  phi_q2_ellipticity(phi_Edge_Lens, q_Edge_Lens)
        centre_X_Edge_Lens = centre_X_Edge_Source + np.random.uniform(-0.25, 0.25)
        centre_Y_Edge_Lens = centre_Y_Edge_Source + np.random.uniform(-0.25, 0.25) #generateLensPos(centre_X_Edge_Source, centre_Y_Edge_Source, theta_E_SIS_Edge)
        gamma1_Edge_Lens = np.random.uniform(-0.05, 0.05)
        gamma2_Edge_Lens = np.random.uniform(-0.05, 0.05)
        

        kwargs_model_composite = {'lens_model_list': ['SIE', 'SHEAR', 'SIE', 'SHEAR'],#, 'SIE', 'SHEAR'],  # list of lens models to be used
                                  'lens_redshift_list': [Mid_lens_Z, Mid_lens_Z, Edge_lens_Z, Edge_lens_Z],#, Edge_lens_Z, Edge_lens_Z],  # list of redshift of the deflections
                                  'lens_light_model_list': ['SERSIC_ELLIPSE', 'SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                                  'source_light_model_list': [ 'SERSIC'],  # list of extended source models to be used
                                  'source_redshift_list': [Edge_source_Z],  # list of redshfits of the sources in same order as source_light_model_list
                                  'cosmo': cosmo,  # astropy.cosmology instance
                                  'z_source_convention': 4.5,  # source redshfit to which the reduced deflections are computed, is the maximal redshift of the ray-tracing
                                  'z_source': 4.5,  # redshift of the default source (if not further specified by 'source_redshift_list') and also serves as the redshift of lensed point sources
                                  }

        # gamma is the shear component 
        kwargs_lens = [
                {'theta_E': theta_E_SIS_Mid, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},  # SIE model
                {'gamma1': gamma1_Mid_Lens, 'gamma2': gamma2_Mid_Lens},  # SHEAR model
                {'theta_E': theta_E_SIS_EdgeML, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}, # SIE model
                {'gamma1': gamma1_Edge_Lens, 'gamma2': gamma2_Edge_Lens}
                ]
        # lens light for JWST
        kwargs_lens_light_mag_F115W = [{'magnitude': Mid_Lens_Gal_Mag_JWST[0], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                       {'magnitude': Edge_Lens_Gal_Mag_JWST[0], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_F150W = [{'magnitude': Mid_Lens_Gal_Mag_JWST[1], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                       {'magnitude': Edge_Lens_Gal_Mag_JWST[1], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_F200W = [{'magnitude': Mid_Lens_Gal_Mag_JWST[2], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                       {'magnitude': Edge_Lens_Gal_Mag_JWST[2], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_F277W = [{'magnitude': Mid_Lens_Gal_Mag_JWST[3], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                       {'magnitude': Edge_Lens_Gal_Mag_JWST[3], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]
        kwargs_lens_light_mag_F356W = [{'magnitude': Mid_Lens_Gal_Mag_JWST[4], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                       {'magnitude': Edge_Lens_Gal_Mag_JWST[4], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]
        kwargs_lens_light_mag_F444W = [{'magnitude': Mid_Lens_Gal_Mag_JWST[5], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                       {'magnitude': Edge_Lens_Gal_Mag_JWST[5], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        # source light
        kwargs_source_mag_F115W = [
                          {'magnitude': Edge_Source_Gal_Mag_JWST[0], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_F150W = [
                          {'magnitude': Edge_Source_Gal_Mag_JWST[1], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_F200W = [
                          {'magnitude': Edge_Source_Gal_Mag_JWST[2], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_F277W = [
                          {'magnitude': Edge_Source_Gal_Mag_JWST[3], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_F356W = [
                          {'magnitude': Edge_Source_Gal_Mag_JWST[4], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_F444W = [
                          {'magnitude': Edge_Source_Gal_Mag_JWST[5], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]

        # lens light Euclid 
        kwargs_lens_light_mag_VIS = [{'magnitude': Mid_Lens_Gal_Mag_Euclid[0], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                     {'magnitude': Edge_Lens_Gal_Mag_Euclid[0], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_Y= [{'magnitude': Mid_Lens_Gal_Mag_Euclid[1], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                  {'magnitude': Edge_Lens_Gal_Mag_Euclid[1], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_H = [{'magnitude': Mid_Lens_Gal_Mag_Euclid[2], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                   {'magnitude': Edge_Lens_Gal_Mag_Euclid[2], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_J = [{'magnitude': Mid_Lens_Gal_Mag_Euclid[3], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                                   {'magnitude': Edge_Lens_Gal_Mag_Euclid[3], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]
        # source light
        kwargs_source_mag_VIS = [
                          {'magnitude': Edge_Source_Gal_Mag_Euclid[0], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_Y = [
                          {'magnitude': Edge_Source_Gal_Mag_Euclid[1], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_H = [
                          {'magnitude': Edge_Source_Gal_Mag_Euclid[2], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_J = [
                          {'magnitude': Edge_Source_Gal_Mag_Euclid[3], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]

    
    
    
    
    
        img_SW, img_LW, image_F115W, image_F150W, image_F200W, image_F277W, image_F356W, image_F444W, img_VIS, img_NISP, image_VIS, image_J, image_Y, image_H = CompoundLensMaker2(kwargs_model_composite, kwargs_lens, kwargs_lens_light_mag_F115W, kwargs_source_mag_F115W,
                                                                                                                                                                               kwargs_lens_light_mag_F150W, kwargs_source_mag_F150W, kwargs_lens_light_mag_F200W, kwargs_source_mag_F200W,
                                                                                                                                                                               kwargs_lens_light_mag_F277W, kwargs_source_mag_F277W, kwargs_lens_light_mag_F356W, kwargs_source_mag_F356W,
                                                                                                                                                                               kwargs_lens_light_mag_F444W, kwargs_source_mag_F444W, kwargs_NIRcam, kwargs_lens_light_mag_VIS,
                                                                                                                                                                               kwargs_source_mag_VIS, kwargs_lens_light_mag_J, kwargs_source_mag_J,
                                                                                                                                                                               kwargs_lens_light_mag_Y, kwargs_source_mag_Y, kwargs_lens_light_mag_H,
                                                                                                                                                                               kwargs_source_mag_H, kwargs_Euclid)
    
    
        countMod = 1
    except ValueError:
        
        Edge_Lens_Gal_Mag_JWST = [9999, 9999, 9999, 9999, 9999, 9999]
        Mid_Lens_Gal_Mag_JWST =  [9999, 9999, 9999, 9999, 9999, 9999]
        Edge_Source_Gal_Mag_JWST = [9999, 9999, 9999, 9999, 9999, 9999]
        Edge_Lens_Gal_Mag_Euclid = [9999, 9999, 9999, 9999]
        Mid_Lens_Gal_Mag_Euclid =  [9999, 9999, 9999, 9999]
        Edge_Source_Gal_Mag_Euclid = [9999, 9999, 9999, 9999]

        theta_E_SIS_Mid = 9999
        theta_E_SIS_EdgeML  = 9999
        theta_E_SIS_EdgeES  = 9999

        R_sersic_Edge_Source = 9999
        n_sersic_Edge_Source = 9999
        centre_X_Edge_Source = 9999
        centre_Y_Edge_Source = 9999

        R_sersic_Mid_Lens = 9999
        n_sersic_Mid_Lens = 9999
        e1_Mid_Lens = 9999
        e2_Mid_Lens = 9999
        centre_X_Mid_Lens = 9999
        centre_Y_Mid_Lens = 9999
        gamma1_Mid_Lens = 9999
        gamma2_Mid_Lens = 9999

        R_sersic_Edge_Lens = 9999
        n_sersic_Edge_Lens = 9999
        e1_Edge_Lens = 9999
        e2_Edge_Lens = 9999
        centre_X_Edge_Lens = 9999
        centre_Y_Edge_Lens = 9999
        gamma1_Edge_Lens = 9999
        gamma2_Edge_Lens = 9999
        
        img_SW =  np.random.random((64, 64, 3))
        img_LW =  np.random.random((64, 64, 3))
        image_F115W =  np.random.random((64, 64))
        image_F150W =  np.random.random((64, 64))
        image_F200W =  np.random.random((64, 64))
        image_F277W =  np.random.random((64, 64))
        image_F356W =  np.random.random((64, 64))
        image_F444W =  np.random.random((64, 64))
        img_VIS = np.random.random((64, 64, 3))
        img_NISP = np.random.random((64, 64, 3))
        image_VIS = np.random.random((64, 64))
        image_J = np.random.random((64, 64))
        image_Y = np.random.random((64, 64))
        image_H = np.random.random((64, 64))
        
        
        countMod = 0
    
    
    arr_params = [Edge_source_Z, Edge_source_Mag, Mid_lens_Z,  Mid_lens_Mag,  Edge_lens_Z, Edge_lens_Mag,  Edge_Source_Gal_Mag_JWST[0], 
                  Edge_Source_Gal_Mag_JWST[1], Edge_Source_Gal_Mag_JWST[2], Edge_Source_Gal_Mag_JWST[3],  Edge_Source_Gal_Mag_JWST[4], Edge_Source_Gal_Mag_JWST[5], 
                  Mid_Lens_Gal_Mag_JWST[0],  Mid_Lens_Gal_Mag_JWST[1], Mid_Lens_Gal_Mag_JWST[2], Mid_Lens_Gal_Mag_JWST[3], Mid_Lens_Gal_Mag_JWST[4], Mid_Lens_Gal_Mag_JWST[5], 
                  Edge_Lens_Gal_Mag_JWST[0], Edge_Lens_Gal_Mag_JWST[1],Edge_Lens_Gal_Mag_JWST[2], Edge_Lens_Gal_Mag_JWST[3], Edge_Lens_Gal_Mag_JWST[4], Edge_Lens_Gal_Mag_JWST[5],
                  Edge_Source_Gal_Mag_Euclid[0], Edge_Source_Gal_Mag_Euclid[1], Edge_Source_Gal_Mag_Euclid[2], Edge_Source_Gal_Mag_Euclid[3],
                  Mid_Lens_Gal_Mag_Euclid[0],  Mid_Lens_Gal_Mag_Euclid[1], Mid_Lens_Gal_Mag_Euclid[2], Mid_Lens_Gal_Mag_Euclid[3], 
                  Edge_Lens_Gal_Mag_Euclid[0], Edge_Lens_Gal_Mag_Euclid[1],Edge_Lens_Gal_Mag_Euclid[2], Edge_Lens_Gal_Mag_Euclid[3],
                  theta_E_SIS_Mid, theta_E_SIS_EdgeML, theta_E_SIS_EdgeES, R_sersic_Edge_Source,  n_sersic_Edge_Source, centre_X_Edge_Source,
                  centre_Y_Edge_Source,  R_sersic_Mid_Lens, n_sersic_Mid_Lens, e1_Mid_Lens, e2_Mid_Lens, centre_X_Mid_Lens, centre_Y_Mid_Lens,
                  gamma1_Mid_Lens, gamma2_Mid_Lens, R_sersic_Edge_Lens, n_sersic_Edge_Lens, e1_Edge_Lens, e2_Edge_Lens, centre_X_Edge_Lens,
                  centre_Y_Edge_Lens, gamma1_Edge_Lens, gamma2_Edge_Lens]
    
    
    
    
    return img_SW, img_LW, image_F115W, image_F150W, image_F200W, image_F277W, image_F356W, image_F444W, img_VIS, img_NISP, image_VIS, image_J, image_Y, image_H, arr_params, countMod


Let's create a single compound lens.

In [24]:
img_SW, img_LW, image_F115W, image_F150W, image_F200W, image_F277W, image_F356W, image_F444W, img_VIS, img_NISP, image_VIS, image_J, image_Y, image_H, arr_params, countMod = CreateCompoundLenses(cosmo=cosmo)


We can make multiple compound lenses. The cGAN expects a *.fits* extension, so we'll save the lenses in a fits file format.

In [25]:
# make many more!!
i = 0
end = 5
# save to YOUR directory
loc = wdir+'Data/Euclid-JWST/'
file_name = '_lens.fits'
while i < end:
    img_SW, img_LW, img_F115W, img_F150W, img_F200W, img_F277W, img_F356W, img_F444W, img_VIS, img_NISP, image_VIS, image_J, image_Y, image_H, arr_params, countMod = CreateCompoundLenses(cosmo)
    name_out = str(i)+file_name
    fits.writeto(loc+'F115W/'+name_out, img_F115W, overwrite=True)
    fits.writeto(loc+'F150W/'+name_out, img_F150W, overwrite=True)
    fits.writeto(loc+'F200W/'+name_out, img_F200W, overwrite=True)
    fits.writeto(loc+'F277W/'+name_out, img_F277W, overwrite=True)
    fits.writeto(loc+'F356W/'+name_out, img_F356W, overwrite=True)
    fits.writeto(loc+'F444W/'+name_out, img_F444W, overwrite=True)
    fits.writeto(loc+'VIS/'+name_out, image_VIS, overwrite=True)
    fits.writeto(loc+'NISP-J/'+name_out, image_J, overwrite=True)
    fits.writeto(loc+'NISP-Y/'+name_out, image_Y, overwrite=True)
    fits.writeto(loc+'NISP-H/'+name_out, image_H, overwrite=True)
    i += countMod
print('Done!')

Done!


Due to the different wavelength ranges of JWST NIRcam and Euclid, Skypy oftentimes throws a wavelength shift error for Euclid-VIS. Check the generated lenses for the except ValueError.

As a result you should see the following output. These are grayscale outputs from the JWST NIRcam short wavelength images, *img_SW*.

![compound lenses](images/Compound_Lenses.png)